In [1]:
import pandas as pd
import re
import warnings

warnings.filterwarnings('ignore')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin

def text_normalize(x):
    return ' '.join(r for r in re.findall(r'[а-я]+', str(x).lower())
                    if len(r) > 2)

def top_pair(values, keys, n=3):
    return sorted(zip(values, keys), reverse=True)[:n]

def predict(model, X):
    predictions = model.predict_proba(X)
    classes = model.classes_
    return [top_pair(p, classes) for p in predictions]

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        if self.key == 'symptomps':
            return data_dict[self.key]
        else:
            return data_dict[self.key].values.reshape(-1, 1)

# Data

In [2]:
train = pd.read_csv(r'train_data.csv', sep=';')
train['symptomps'] = train['Жалобы'].map(text_normalize)

In [3]:
train.shape

(61976, 11)

In [4]:
train['gender'] = train['Пол'].map(lambda x: 'мужской' if x==1 else 'женский')
train['age'] = train['Возраст'].astype(str).values

In [5]:
train.head()

,Id_Записи,Id_Пациента,Возраст,Диагноз,Жалобы,Источник_рекламы,Клиника,Код_диагноза,Пол,Услуга,symptomps,gender,age
0,0,115819,54,Гипертензивная болезнь сердца [гипертоническая...,"на повышение ад утром до 140/90 мм.рт.ст., пер...",Другое,5,I11,2,"Прием врача-кардиолога повторный, амбулаторный",повышение утром периодич головокружение,женский,54
1,1,399973,32,Доброкачественное новообразование молочной железы,На наличие опухоли в левой молочной железе,Другое,3,D24,2,"Прием врача-онколога (маммолога), повторный, а...",наличие опухоли левой молочной железе,женский,32
2,2,427563,72,Простой хронический бронхит,Активных жалоб нет.,Интернет,6,J41.0,2,Прием первичный врача-пульмонолога,активных жалоб нет,женский,72
3,3,257197,55,Другая дорсалгия,"на сохраняющиеся боли в спине и пояснице, сков...",Другое,3,M54.8,1,"Прием врача-невролога повторный, амбулаторный",сохраняющиеся боли спине пояснице скованность ней,мужской,55
4,4,281066,28,Острый фарингит,"на дискомфорт в горле, слабое першение, слабость",Другое,3,J02,2,"Прием врача-оториноларинголога повторный, амбу...",дискомфорт горле слабое першение слабость,женский,28


In [6]:
diag = pd.read_pickle('diagnoz_vrach.pickle')

In [7]:
train = train[train['Код_диагноза'].isin(diag.keys())]

In [8]:
train.shape

(44819, 13)

In [9]:
# Remove rare diseases

In [10]:
train['Диагноз'].value_counts()

Острая инфекция верхних дыхательных путей неуточненная                   2147
Остеохондроз позвоночника у взрослых                                     1949
Острый назофарингит (насморк)                                            1379
Беременность подтвержденная                                              1358
Хронический простатит                                                    1111
                                                                         ... 
Другая уточненная форма острой диссеминированной демиелинизации             1
Сходящееся содружественное косоглазие                                       1
Кристаллические отложения в стекловидном теле                               1
Амилоидоз                                                                   1
Хронический активный гепатит, не классифицированный в других рубриках       1
Name: Диагноз, Length: 783, dtype: int64

In [11]:
t = train['Диагноз'].value_counts()
t = t[t <= 50]

In [12]:
train = train[~train['Диагноз'].isin(t.index)]

In [13]:
train.shape

(38948, 13)

# Pipeline

## Simple train/test split

In [15]:
test = train.sample(frac=0.1, random_state=0)
y_test = test['Диагноз'].values 

In [16]:
X = train[~train['Id_Записи'].isin(test['Id_Записи'].unique())]
y = train[~train['Id_Записи'].isin(test['Id_Записи'].unique())]['Диагноз'].values

## Model

In [14]:
clf = CalibratedClassifierCV(
                base_estimator=svm.SVC(kernel='linear', 
                                       C=.1, probability=False),
                method='isotonic')

In [17]:
model = Pipeline([
        ('union', FeatureUnion(
        transformer_list=[

        # Pipeline for pulling features
            ('tfidf', Pipeline([
                        ('selector', ItemSelector(key='symptomps')),
                        ('tdidf', TfidfVectorizer(min_df=10))
            ])),
            ('age', Pipeline([
                        ('selector', ItemSelector(key='age')),
                        ('ohe', OneHotEncoder(handle_unknown='ignore'))
            ])),
            ('gender', Pipeline([
                        ('selector', ItemSelector(key='gender')),
                        ('ohe', OneHotEncoder(handle_unknown='ignore'))
            ])),
        ],

        # weight components in FeatureUnion
        transformer_weights={
            'tfidf': 0.8,
            'age': 0.1,
            'gender': 0.1
        },
    )),
        ('svc', clf)])

model.fit(X, y)

Pipeline(memory=None,
         steps=[('union',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('tfidf',
                                                 Pipeline(memory=None,
                                                          steps=[('selector',
                                                                  ItemSelector(key='symptomps')),
                                                                 ('tdidf',
                                                                  TfidfVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.float64'>,
                                                                                  encoding='utf-8',
                

In [18]:
predict(model, X[:10])

[[(0.45837015817119636,
   'Гипертензивная болезнь сердца [гипертоническая болезнь сердца с преимущественным поражением сердца]'),
  (0.07071038757234079, 'Другие уточненные поражения сосудов мозга'),
  (0.04619110409643721,
   'Расстройства вегетативной [автономной] нервной системы')],
 [(0.26424998979079867, 'Общий медицинский осмотр'),
  (0.1009455479241884,
   'Острая инфекция верхних дыхательных путей неуточненная'),
  (0.04552352290765432,
   'Гипертензивная [гипертоническая] болезнь с преимущественным поражением сердца без (застойной) сердечной недостаточности')],
 [(0.36358363250136566, 'Другая дорсалгия'),
  (0.2727643457523841, 'Другие уточненные дорсопатии'),
  (0.10723722412137981, 'Боль внизу спины')],
 [(0.44978027236002366, 'Хронический тонзиллит'),
  (0.08251639222131657, 'Хронический фарингит'),
  (0.08036420822965283,
   'Острая инфекция верхних дыхательных путей неуточненная')],
 [(0.21558791630987836, 'Остеохондроз позвоночника у взрослых'),
  (0.09004945798394458, 

# Save model

In [19]:
from joblib import dump, load
dump(model, 'model/mymed_v0.joblib') 

['model/mymed_v0.joblib']

# Evaluate performance

In [22]:
def top_elements(a, n=5):
    return sorted(range(len(a)), key=lambda i: a[i])[-n:]

def check_precision(model, X, y):
    predictions = model.predict_proba(X)
    classes = model.classes_
    return sum([1 if true in classes[top_elements(prediction)] else 0 for true, prediction in zip(y, predictions)]) / len(y)

In [23]:
model = load('model/mymed_v0.joblib')

In [24]:
print('Precision at top 5: {0:0.2f}'.format(check_precision(model, test, y_test)))

Precision at top 5: 0.68


# Map predictions to doctors

In [25]:
train.head()

,Id_Записи,Id_Пациента,Возраст,Диагноз,Жалобы,Источник_рекламы,Клиника,Код_диагноза,Пол,Услуга,symptomps,gender,age
0,0,115819,54,Гипертензивная болезнь сердца [гипертоническая...,"на повышение ад утром до 140/90 мм.рт.ст., пер...",Другое,5,I11,2,"Прием врача-кардиолога повторный, амбулаторный",повышение утром периодич головокружение,женский,54
2,2,427563,72,Простой хронический бронхит,Активных жалоб нет.,Интернет,6,J41.0,2,Прием первичный врача-пульмонолога,активных жалоб нет,женский,72
3,3,257197,55,Другая дорсалгия,"на сохраняющиеся боли в спине и пояснице, сков...",Другое,3,M54.8,1,"Прием врача-невролога повторный, амбулаторный",сохраняющиеся боли спине пояснице скованность ней,мужской,55
4,4,281066,28,Острый фарингит,"на дискомфорт в горле, слабое першение, слабость",Другое,3,J02,2,"Прием врача-оториноларинголога повторный, амбу...",дискомфорт горле слабое першение слабость,женский,28
6,6,416352,29,Поражение межпозвоночных дисков других отделов,Не изменились с момента первого приема,Интернет,2,M51,2,"Прием врача-невролога повторный, амбулаторный",изменились момента первого приема,женский,29


In [26]:
train = pd.read_csv(r'train_data.csv', sep=';')

In [27]:
docs = pd.DataFrame({'Код_диагноза' : list(diag.keys()), 'Доктор' : list(diag.values())})

In [28]:
docs.shape[0]

787

In [29]:
docs = pd.merge(docs, train[['Код_диагноза', 'Диагноз']].drop_duplicates(),
               how='inner', on='Код_диагноза')

In [30]:
docs.shape[0]

787

In [31]:
pcp_dict = docs.set_index('Диагноз')['Доктор'].to_dict()

In [32]:
dump(model, 'model/pcp_dict.joblib') 

['model/pcp_dict.joblib']

# Final predict function

In [33]:
prediction = pd.DataFrame(predict(model, X.iloc[:1,:])[0], 
                          columns = ['Вероятность', 'Болезнь'])

In [34]:
prediction['Доктор'] = prediction['Болезнь'].map(pcp_dict)

In [35]:
prediction

,Вероятность,Болезнь,Доктор
0,0.458370,Гипертензивная болезнь сердца [гипертоническая...,терапевт
1,0.070710,Другие уточненные поражения сосудов мозга,невролог
2,0.046191,Расстройства вегетативной [автономной] нервной...,невролог
